In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [3]:
import os 
os.chdir("/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/EraV2_Transformers")
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/EraV2_Transformers


In [9]:
from nlp_models.classic_transformer import build_transformer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import torch.functional as F

In [5]:
d_model = 512
heads = 8
N = 6
d_ff = 2048
dropout_rate = 0.1
dim = 11

In [6]:
transformer_model = build_transformer(d_model, heads, N, d_ff, dropout_rate, dim)

MaskedMultiHeadAttentionBlock(
  (Wq): Linear(in_features=512, out_features=512, bias=False)
  (Wk): Linear(in_features=512, out_features=512, bias=False)
  (Wv): Linear(in_features=512, out_features=512, bias=False)
  (Wo): Linear(in_features=512, out_features=512, bias=False)
  (dropout): Dropout(p=0.1, inplace=False)
)
FeedForwardBlock(
  (linear1): Linear(in_features=512, out_features=2048, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=2048, out_features=512, bias=True)
)
EncoderBlock(
  (attention_block): MaskedMultiHeadAttentionBlock(
    (Wq): Linear(in_features=512, out_features=512, bias=False)
    (Wk): Linear(in_features=512, out_features=512, bias=False)
    (Wv): Linear(in_features=512, out_features=512, bias=False)
    (Wo): Linear(in_features=512, out_features=512, bias=False)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (feed_forward_block): FeedForwardBlock(
    (linear1): Linear(in_features=512, out_features=2048, bias=Tru

In [10]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)
    return subsequent_mask == 0

In [11]:
def model_inference_test():
    transformer_model.eval()

    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    src_mask = torch.ones(1, 1, 10)

    print(src.shape)
    print(src_mask.shape)

    memory = transformer_model.encode(src, src_mask)
    print(memory.shape)

    # output placeholder starts with 0
    ys = torch.zeros(1, 1).type_as(src)

    for i in range(9):

        # output of the decoder
        out = transformer_model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        print(out.shape)
        # output of the linear layer in logits
        prob = transformer_model.linear_projection(out[:, -1])
        print(prob.shape)
        # get the next word by argmax
        next_word = torch.argmax(prob, dim=1)
        print(next_word)
        # 0 dim next word
        next_word = next_word.data[0]
        # concat the next word to the output
        ys = torch.cat(
            [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )

    print(ys)

In [13]:
model_inference_test()

torch.Size([1, 10])
torch.Size([1, 1, 10])
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
torch.Size([1, 10, 512])
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores


Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
torch.Size([1, 1, 512])
torch.Size([1, 11])
tensor([1])
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
Applying Dropout to the softmax scores
Applying Mask to the scaled_qk
App